In [1]:
import re
from collections import defaultdict, Counter
import numpy as np
import jieba  
import pandas as pd

In [2]:
def clean_text(text):
    """
    Clean a single text: retain Chinese characters, common punctuation, remove HTML tags, ads, etc.
    """
    if not isinstance(text, str):
        return ""
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Retain Chinese characters, common punctuation, numbers, letters (optional)
    cleaned = re.sub(r'[^\u4e00-\u9fa5，。！？；：、‘’“”（）【】\n\s0-9a-zA-Z]', '', text)
    # Remove extra spaces and newlines
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

def preprocess_corpus(df):
    """
    Preprocess the entire DataFrame and return a list containing all cleaned texts.
    """
    corpus = []
    for idx, row in df.iterrows():
        title_clean = clean_text(row['文本标题'])
        content_clean = clean_text(row['文本内容'])
        # Merge the title and content and separate them with spaces
        full_text = f"{title_clean} {content_clean}"
        corpus.append(full_text)
    return corpus

In [3]:
def calculate_cooccurrence_prob(corpus, window_size=2):
    """
    Calculate the probability of any two Chinese characters co-occurring adjacently P(c_j | c_i)
    Returns a dictionary: {(c_i, c_j): probability}
    """
    char_count = defaultdict(int)      # Character frequency
    cooccur_count = defaultdict(lambda: defaultdict(int))  # Co-occurrence frequency

    for text in corpus:
        # Iterate over each character
        for i in range(len(text) - window_size + 1):
            window = text[i:i+window_size]
            if len(window) == window_size:
                c1, c2 = window[0], window[1]
                char_count[c1] += 1
                cooccur_count[c1][c2] += 1

    # Calculate the conditional probability P(c_j | c_i)
    prob_dict = {}
    for c1 in cooccur_count:
        for c2 in cooccur_count[c1]:
            prob_dict[(c1, c2)] = cooccur_count[c1][c2] / char_count[c1]

    return prob_dict, char_count

In [4]:
def calculate_pmi(cooccur_prob_dict, char_count, total_chars):
    """
    Calculate pointwise mutual information PMI(c_i, c_j) = log2( P(c_i, c_j) / (P(c_i) * P(c_j)) )
    Note: Here P(c_i, c_j) = P(c_j|c_i) * P(c_i)
    """
    pmi_dict = {}
    for (c1, c2), cooccur_prob in cooccur_prob_dict.items():
        p_c1 = char_count[c1] / total_chars
        p_c2 = char_count[c2] / total_chars
        # Joint probability P(c1,c2) = P(c2|c1) * P(c1)
        p_joint = cooccur_prob * p_c1
        if p_c1 > 0 and p_c2 > 0 and p_joint > 0:
            pmi = np.log2(p_joint / (p_c1 * p_c2))
            pmi_dict[(c1, c2)] = pmi
    return pmi_dict

In [5]:
def article_to_vector(article_text, vocab):
    """
    Convert the article into a word frequency vector (based on the given vocabulary list)
    """
    # use jieba for tokenization
    words = jieba.lcut(article_text)
    
    word_count = Counter(words)
    vector = np.array([word_count.get(word, 0) for word in vocab])
    # Normalize to probability distribution
    if np.sum(vector) > 0:
        vector = vector / np.sum(vector)
    return vector


def avg_mutual_information_precise(article1, article2, vocab):
    """
    To calculate the average mutual information between two articles more precisely (a joint distribution needs to be constructed)
    This method calculates the "average mutual information at the lexical level".
    """
    vec1 = article_to_vector(article1, vocab)
    vec2 = article_to_vector(article2, vocab)
    
    # Construct joint distribution (simplified: assume words are independent, but they are not)
    # The true joint distribution requires counting the co-occurrence of each pair of words in the two articles.
    # Since we only have two articles, we cannot construct a reliable joint distribution.
    # Therefore, we switch to calculating the "average pointwise mutual information of word co-occurrence".
    
    # Extract all words from the two articles
    words1 = set(jieba.lcut(article1) if 'jieba' in globals() else list(article1))
    words2 = set(jieba.lcut(article2) if 'jieba' in globals() else list(article2))
    
    # Calculate the PMI of the intersecting words (requires global corpus statistics)
    # Since we do not have a global corpus, here we can only provide a framework.
    # In actual applications, the global char_count and cooccur_count calculated earlier should be used.
    
    # We output a simple metric based on word overlap
    overlap = len(words1 & words2) / len(words1 | words2)  # Jaccard similarity
    return overlap


In [6]:
def main_pipeline(df):
    """
    Main flow function: From the original DataFrame to all calculation results
    """
    print("Starting data cleaning...")
    corpus = preprocess_corpus(df)
    print(f"Cleaning completed, total {len(corpus)} articles")

    print("\nStarting calculation of co-occurrence probabilities...")
    cooccur_prob, char_count = calculate_cooccurrence_prob(corpus)
    total_chars = sum(char_count.values())
    print(f"Total {len(char_count)} unique Chinese characters counted")

    print("\nStarting calculation of pointwise mutual information...")
    pmi_result = calculate_pmi(cooccur_prob, char_count, total_chars)
    print(f"Calculated PMI for {len(pmi_result)} pairs of Chinese characters")

    print("\nStarting calculation of average mutual information between articles...")
    # Calculate the average mutual information between the first two articles as an example
    if len(corpus) >= 2:
        article1 = corpus[0]
        article2 = corpus[1]
        vocab = list(set(list(article1) + list(article2)))
        avg_mi = avg_mutual_information_precise(article1, article2, vocab)
        print(f"Average mutual information between Article 1 and Article 2 (Jaccard approximation): {avg_mi:.6f}")
    else:
        print("Insufficient number of articles to calculate inter-article mutual information")

    # Return all results for further analysis or saving
    return {
        'corpus': corpus,
        'cooccur_prob': cooccur_prob,
        'char_count': char_count,
        'pmi_result': pmi_result,
        'total_chars': total_chars
    }



In [7]:
if __name__=='__main__':
    df = pd.read_csv('./data/chinanews.csv')
    res = main_pipeline(df)

Starting data cleaning...
Cleaning completed, total 965 articles

Starting calculation of co-occurrence probabilities...


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/v2/ptjfpzrd3pz9s7p0353vdcvw0000gn/T/jieba.cache


Total 3968 unique Chinese characters counted

Starting calculation of pointwise mutual information...
Calculated PMI for 165590 pairs of Chinese characters

Starting calculation of average mutual information between articles...


Loading model cost 0.248 seconds.
Prefix dict has been built successfully.


Average mutual information between Article 1 and Article 2 (Jaccard approximation): 0.061090


In [8]:
import json
import pandas as pd
import numpy as np
from datetime import datetime

def convert_numpy_types(obj):
    """
    Recursively convert numpy data types to native Python types
    """
    if isinstance(obj, (np.integer, np.int64, np.int32, np.int16, np.int8)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32, np.float16)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, dict):
        return {str(k): convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, np.generic):
        return obj.item()  # For other numpy types
    else:
        return obj

def save_results_separate(results, base_filename):
    """
    Save different components in appropriate formats (fixed version)
    """
    
    # 1. Save corpus (text data)
    if 'corpus' in results and results['corpus'] is not None:
        try:
            with open(f'./data/{base_filename}_corpus.txt', 'w', encoding='utf-8') as f:
                if isinstance(results['corpus'], list):
                    for item in results['corpus']:
                        f.write(str(item) + '\n')
                else:
                    f.write(str(results['corpus']))
            print(f"✓ Saved corpus to: ./data/{base_filename}_corpus.txt")
        except Exception as e:
            print(f"✗ Failed to save corpus: {e}")
    
    # 2. Save cooccur_prob
    if 'cooccur_prob' in results and results['cooccur_prob'] is not None:
        try:
            if hasattr(results['cooccur_prob'], 'to_csv'):
                results['cooccur_prob'].to_csv(f'./data/{base_filename}_cooccur_prob.csv', encoding='utf-8')
                print(f"✓ Saved cooccur_prob to: ./data/{base_filename}_cooccur_prob.csv")
            elif isinstance(results['cooccur_prob'], (pd.DataFrame, np.ndarray)):
                pd.DataFrame(results['cooccur_prob']).to_csv(f'./data/{base_filename}_cooccur_prob.csv')
                print(f"✓ Saved cooccur_prob to: ./data/{base_filename}_cooccur_prob.csv")
            else:
                # If it's a dictionary or other type
                converted_data = convert_numpy_types(results['cooccur_prob'])
                with open(f'./data/{base_filename}_cooccur_prob.json', 'w', encoding='utf-8') as f:
                    json.dump(converted_data, f, ensure_ascii=False, indent=2)
                print(f"✓ Saved cooccur_prob to: ./data/{base_filename}_cooccur_prob.json")
        except Exception as e:
            print(f"✗ Failed to save cooccur_prob: {e}")
    
    # 3. Save char_count (dictionary data)
    if 'char_count' in results and results['char_count'] is not None:
        try:
            converted_char_count = convert_numpy_types(results['char_count'])
            with open(f'./data/{base_filename}_char_count.json', 'w', encoding='utf-8') as f:
                json.dump(converted_char_count, f, ensure_ascii=False, indent=2)
            print(f"✓ Saved char_count to: ./data/{base_filename}_char_count.json")
        except Exception as e:
            print(f"✗ Failed to save char_count: {e}")
    
    # 4. Save pmi_result (fixed version)
    if 'pmi_result' in results and results['pmi_result'] is not None:
        try:
            # First convert numpy types
            converted_pmi = convert_numpy_types(results['pmi_result'])
            
            # Save as JSON
            with open(f'./data/{base_filename}_pmi_result.json', 'w', encoding='utf-8') as f:
                json.dump(converted_pmi, f, ensure_ascii=False, indent=2)
            print(f"✓ Saved pmi_result to: ./data/{base_filename}_pmi_result.json")
            
            # Also save as CSV for easier viewing
            if isinstance(converted_pmi, dict):
                pmi_df = pd.DataFrame.from_dict(converted_pmi, orient='index', columns=['PMI_Value'])
                pmi_df.index.name = 'Character_Pair'
                pmi_df.to_csv(f'./data/{base_filename}_pmi_result.csv', encoding='utf-8')
                print(f"✓ Saved pmi_result to: ./data/{base_filename}_pmi_result.csv")
                
        except Exception as e:
            print(f"✗ Failed to save pmi_result: {e}")
    
    # 5. Save total_chars and other numeric values
    if 'total_chars' in results and results['total_chars'] is not None:
        try:
            metadata = {
                'total_chars': convert_numpy_types(results.get('total_chars')),
                'save_time': datetime.now().isoformat(),
                'data_types': {
                    'corpus': type(results.get('corpus')).__name__,
                    'cooccur_prob': type(results.get('cooccur_prob')).__name__,
                    'char_count': type(results.get('char_count')).__name__,
                    'pmi_result': type(results.get('pmi_result')).__name__
                }
            }
            
            with open(f'./data/{base_filename}_metadata.json', 'w', encoding='utf-8') as f:
                json.dump(metadata, f, ensure_ascii=False, indent=2)
            print(f"✓ Saved metadata to: ./data/{base_filename}_metadata.json")
            
        except Exception as e:
            print(f"✗ Failed to save metadata: {e}")
    
    print(f"\n🎉 Results have been saved to separate {base_filename}_* files")


save_results_separate(res, 'chinanews_analysis_results')

✓ Saved corpus to: ./data/chinanews_analysis_results_corpus.txt
✓ Saved cooccur_prob to: ./data/chinanews_analysis_results_cooccur_prob.json
✓ Saved char_count to: ./data/chinanews_analysis_results_char_count.json
✓ Saved pmi_result to: ./data/chinanews_analysis_results_pmi_result.json
✓ Saved pmi_result to: ./data/chinanews_analysis_results_pmi_result.csv
✓ Saved metadata to: ./data/chinanews_analysis_results_metadata.json

🎉 Results have been saved to separate chinanews_analysis_results_* files


In [9]:
def save_pmi_result_special(pmi_result, filename):
    """
    A function specifically for saving PMI results
    """
    # Convert numpy types
    converted_pmi = convert_numpy_types(pmi_result)
    
    # Save as JSON
    with open(f'./data/{filename}.json', 'w', encoding='utf-8') as f:
        json.dump(converted_pmi, f, ensure_ascii=False, indent=2)
    
    # Save as sorted CSV
    if isinstance(converted_pmi, dict):
        # Sort by PMI value
        sorted_pmi = sorted(converted_pmi.items(), key=lambda x: x[1], reverse=True)
        
        # Create DataFrame
        pmi_df = pd.DataFrame(sorted_pmi, columns=['Character_Pair', 'PMI_Value'])
        pmi_df.to_csv(f'./data/{filename}.csv', index=False, encoding='utf-8')
        
        # Also save top N highest PMI values
        top_n = min(50, len(sorted_pmi))
        top_pmi = sorted_pmi[:top_n]
        
        print(f"Top {top_n} PMI values:")
        for i, (pair, value) in enumerate(top_pmi, 1):
            print(f"{i:2d}. {pair}: {value:.4f}")
    
    print(f"PMI results have been saved to: ./data/{filename}.json and ./data/{filename}.csv")
# Specifically save PMI results
save_pmi_result_special(res['pmi_result'], 'chinanews_pmi_analysis')

Top 50 PMI values:
 1. ('妊', '娠'): 21.1805
 2. ('踌', '躇'): 21.1805
 3. ('呋', '喹'): 21.1805
 4. ('浐', '灞'): 21.1805
 5. ('纐', '纈'): 21.1805
 6. ('锱', '铢'): 21.1805
 7. ('袈', '裟'): 21.1805
 8. ('哀', '悼'): 21.1805
 9. ('秸', '秆'): 21.1805
10. ('胫', '腓'): 21.1805
11. ('匍', '匐'): 21.1805
12. ('饥', '馑'): 21.1805
13. ('涅', '槃'): 21.1805
14. ('簸', '箕'): 21.1805
15. ('蜉', '蝣'): 21.1805
16. ('徜', '徉'): 21.1805
17. ('疙', '瘩'): 21.1805
18. ('瘴', '痼'): 21.1805
19. ('壬', '珅'): 21.1805
20. ('亵', '渎'): 21.1805
21. ('鸳', '鸯'): 21.1805
22. ('褶', '皱'): 21.1805
23. ('忐', '忑'): 21.1805
24. ('咯', '噔'): 21.1805
25. ('彷', '徨'): 21.1805
26. ('粱', '饴'): 21.1805
27. ('凛', '冽'): 21.1805
28. ('猖', '獗'): 21.1805
29. ('鹦', '鹉'): 21.1805
30. ('醜', '斝'): 21.1805
31. ('耄', '耋'): 21.1805
32. ('篠', '塚'): 21.1805
33. ('踟', '蹰'): 21.1805
34. ('恍', '惚'): 20.1805
35. ('雇', '佣'): 20.1805
36. ('仝', '仺'): 20.1805
37. ('遐', '迩'): 20.1805
38. ('昳', '昤'): 20.1805
39. ('朦', '胧'): 20.1805
40. ('氤', '氲'): 20.1805
41. ('徘', '徊'): 20.18

In [10]:
def analyze_pmi_results(pmi_result, top_k=20):
    """
    Analyze and print the PMI results
    """
    converted_pmi = convert_numpy_types(pmi_result)
    
    if isinstance(converted_pmi, dict):
        # Sort by PMI value
        sorted_pmi = sorted(converted_pmi.items(), key=lambda x: x[1], reverse=True)
        
        print(f"PMI analysis results (top {top_k}):")
        print("=" * 50)
        print(f"{'Rank':<4} {'Character Pair':<10} {'PMI Value':<10}")
        print("-" * 50)
        
        for i, (pair, value) in enumerate(sorted_pmi[:top_k], 1):
            print(f"{i:<4} {str(pair):<10} {value:.4f}")
        
        # Some basic statistics
        values = [v for _, v in sorted_pmi]
        print("\nStatistics:")
        print(f"Total character pairs: {len(sorted_pmi)}")
        print(f"Highest PMI value: {max(values):.4f}")
        print(f"Lowest PMI value: {min(values):.4f}")
        print(f"Average PMI value: {np.mean(values):.4f}")
        print(f"Median PMI value: {np.median(values):.4f}")

# Analyze PMI results
analyze_pmi_results(res['pmi_result'], top_k=20)

PMI analysis results (top 20):
Rank Character Pair PMI Value 
--------------------------------------------------
1    ('妊', '娠') 21.1805
2    ('踌', '躇') 21.1805
3    ('呋', '喹') 21.1805
4    ('浐', '灞') 21.1805
5    ('纐', '纈') 21.1805
6    ('锱', '铢') 21.1805
7    ('袈', '裟') 21.1805
8    ('哀', '悼') 21.1805
9    ('秸', '秆') 21.1805
10   ('胫', '腓') 21.1805
11   ('匍', '匐') 21.1805
12   ('饥', '馑') 21.1805
13   ('涅', '槃') 21.1805
14   ('簸', '箕') 21.1805
15   ('蜉', '蝣') 21.1805
16   ('徜', '徉') 21.1805
17   ('疙', '瘩') 21.1805
18   ('瘴', '痼') 21.1805
19   ('壬', '珅') 21.1805
20   ('亵', '渎') 21.1805

Statistics:
Total character pairs: 165590
Highest PMI value: 21.1805
Lowest PMI value: -9.1123
Average PMI value: 2.1507
Median PMI value: 1.6918
